# Load file

In [98]:
# Load excel file that contains two columns 

#-----------------------------------------------------------
# Source Col            ;           Target Col
# 'Utilities'           ;   'Other manufacturing equipment'
# 'Other stuff'         ;   'Other manufacturing equipment'
#-----------------------------------------------------------

In [99]:
from IPython.display import Image

In [3]:
import pandas as pd

In [4]:
PATH_TO_EXCEL = "../data/......"

# Read excel file with header = row number 1
#df = pd.read_excel(PATH_TO_EXCEL, header=0)

# pre-process source and target

In [5]:
# Potential pre-processing steps
# (1) Stemming
# (2) Lemmatization
# (3) Lowercase
# (4) Delete Organizational entities from accounts
# (5) Delete numbers
# (6) Harmonize words
# (7) ...

In [6]:
import spacy

In [7]:
class PreProcessor(object):
    """ Pre-processes a given text string using NLTK and spacy library
    
    """
    
    def __init__(self, text, language="english"):
        """ Loading text and spacy model
        
        :param text: A raw string of words or sentences or corpus
        :param language: Default 'english'. Can be switched to 'german'
        """
        
        self.text = text
        self.language = language
        
        if not ((self.language is "english") or (self.language is "german")):
            raise Exception("Please provide valid language parameter")
            return None
            
    def _text2spacyToken(self, text):
        if self.language is "english":
            nlp = spacy.load("en")
        elif self.language is "german":
            nlp = spacy.load("de")
            
        doc = nlp(text)
        
        return doc.text
        
    def stemming(self, stemmer="SnowballStemmer"):
        
        stemmedText = _text2spacyToken(self.text)
        return stemmedText        
            
    def test(self):
        pass
    

In [8]:
processed = PreProcessor("Hello world", language="german")

In [9]:
txt = processed.stemming

In [10]:
txt

<bound method PreProcessor.stemming of <__main__.PreProcessor object at 0x111b704a8>>

In [263]:
import pandas as pd
import spacy
from nltk.stem import PorterStemmer

In [264]:
nlp = spacy.load("en")
stemmer = PorterStemmer()

In [35]:
df = pd.read_excel("../data/coa_target_src.xlsx")
df.head()

,PRC (Source),IFRS (Target),Unnamed: 2,Unnamed: 3
0,Level1,Level1,Level2,Level3
1,Current assets:,Intangible assets,Industrial rights and similiar rights,NaN
2,Currency funds,Intangible assets,Trademarks,NaN
3,Financial assets measured at fair value throu...,Intangible assets,Patents,NaN
4,Derivative financial assets,Intangible assets,Goodwill,NaN


In [67]:
import re

def cleanText(text):
    text = str(text)
    text = text.rstrip().lstrip()
    text = re.sub("\n", "", text)
    text = re.sub("\s+", " ", text)
    text = re.sub("\/", " ", text)
    text = re.sub("[^A-Za-zäöüÄÖÜß\s]", "", text)
    return text

In [68]:
sourceDf = [cleanText(row) for row in df.iloc[1:,0]]

In [256]:
targetDF = list(set([cleanText(row) for row in df.iloc[1:,2]]))

## Preprocess text

In [274]:
def preprocess(text, lower=True, lemma=False, stemming=False):
    """
    
    """
    if lemma:
        doc = nlp(text)
        text = " ".join([token.lemma_ for token in doc])
        
    if stemming:
        text = stemmer.stem(text)
        
    if lower:
        text = text.lower()
    
    return text

In [277]:
preprocess("This is an accounting receivables related topic that I want to solve",
           lemma=True,
          stemming=True)

'this be a account receivable relate topic that -pron- want to solv'

# Recommender systems

## n-gram (char level) recommender (Jaccard)

In [73]:
from nltk import ngrams

In [85]:
def nGramCharTransformer(token, n=2, padRight = False, padLeft = False):
    """ 
    
    :token: String containing text
    :n: Parameter that specifies the number of nGrams
    
    :returns: Returns a list of nGrams
    """
    
    return list(ngrams(token, n, pad_right=padRight, pad_left=padLeft))
    

In [88]:
for i, tar in enumerate(targetDF):
    print(i, tar)

0 Trade payables to affiliated related companies
1 Trade receivables from third parties
2 Land and land rights
3 Income tax assets
4 Other short term deposits
5 Other financial liabilities  employees
6 VAT domestic
7 Advanced payments
8 Financial liabilities to shareholders
9 Goodwill
10 Restricted cash excluded from cash and cash equivalents
11 Prepayments to suppliers
12 Restricted cash included inc ash can cash equivalents
13 Capital reserve
14 Technical equipment and machinery
15 Other financial liabilities to affiliated related companies
16 Tooling
17 Provisions for taxes
18 Other liabilities to affiliated related companies
19 Buildings and factories
20 Trademarks
21 Capitalized development costs
22 Shortterm securities
23 Work in progress
24 Deferred income
25 Cash in hand
26 Other equipment and office equipment
27 Vehicles
28 Financial receivables from affiliated companies
29 Trade receivables  gross
30 Raw materials and manufacturing supplies
31 Deferred tax assets
32 Other pro

In [136]:
sourceDf[0]

'Current assets'

In [137]:
targetDF[52]

'Other assets'

In [138]:
src1 = nGramCharTransformer(sourceDf[0])
print(src1)

[('C', 'u'), ('u', 'r'), ('r', 'r'), ('r', 'e'), ('e', 'n'), ('n', 't'), ('t', ' '), (' ', 'a'), ('a', 's'), ('s', 's'), ('s', 'e'), ('e', 't'), ('t', 's')]


In [139]:
tar1 = nGramCharTransformer(targetDF[52])
print(tar1)

[('O', 't'), ('t', 'h'), ('h', 'e'), ('e', 'r'), ('r', ' '), (' ', 'a'), ('a', 's'), ('s', 's'), ('s', 'e'), ('e', 't'), ('t', 's')]


In [140]:
Image(url= "https://wikimedia.org/api/rest_v1/media/math/render/svg/eaef5aa86949f49e7dc6b9c8c3dd8b233332c9e7", width=300, height=300)

In [165]:
def jaccardCoefficient(A, B):
    """ Defined as the size of the intersection 
    divided by the size of the union of the sample sets
    
    :param A: 1st List of items
    :param B: 2nd List of items
    
    :returns: Returns floating point similarity score 
    """
    
    A = set(A)
    B = set(B)
    
    intersection = len(A.intersection(B))
    union = len(A.union(B))
    
    if union > 0:
        return intersection / union
    else:
        return 0

In [166]:
Image(url="https://wikimedia.org/api/rest_v1/media/math/render/svg/593c9e7d96dc41cc82abdb13d3efb9b3b4d6cb51", width=300, height=300)

In [198]:
def tverskyIndex(A, B, alpha=0.5, beta=0.5):
    """ Asymmetric similarity measure on sets 
    that compares a variant to a prototype.
    For sets A and B the Tversky index is a number between 0 and 1.
    
    :param A: 1st List of items
    :param B: 2nd List of items
    :param alpha: Setting alpha=beta=1 produces the Tanimoto coefficient
    :param beta: Setting alpha=beta=0.5 produces Dice's coefficient.
    
    :returns: Returns floating point similarity score 
    """
    
    A = set(A)
    B = set(B)
    
    intersection = len(A.intersection(B))
    
    tverskyIndex = intersection / (intersection + (alpha * len(A-B)) + (beta * len(B-A)))
    
    return tverskyIndex

In [245]:
def getTOPLresults(scores, n=5):
    return sorted(scores, key=lambda scores: scores[1], reverse=True)[:n]

In [215]:
from collections import defaultdict

In [254]:
bigram = defaultdict(list)
N=3

for source in sourceDf:
    sourceNgram = nGramCharTransformer(source, n=N)
    scores = []
    for target in targetDF:
        targetNgram = nGramCharTransformer(target, n=N)
        score = jaccardCoefficient(sourceNgram, targetNgram)
        scores.append((target, score))
            
    TOPL = getTOPLresults(scores) 
    
    bigram[source].append(TOPL)

In [255]:
for k, v in bigram.items():
    print("="*50)
    print(k + "\n")
    for item in v[0]:
        print(item)

Current assets

('Other assets', 0.29411764705882354)
('Deferred tax assets', 0.2608695652173913)
('Income tax assets', 0.22727272727272727)
('Other financial assets', 0.18518518518518517)
('Allowance on other assets', 0.16666666666666666)
Currency funds

('Other noncurrent financial instruments', 0.06818181818181818)
('Deferred income', 0.041666666666666664)
('Deferred tax assets', 0.03571428571428571)
('Deferred tax liabilities', 0.030303030303030304)
('Deferred expenses and accrued income', 0.022727272727272728)
Financial assets measured at fair value through profit or loss for the current period

('Other financial assets', 0.16091954022988506)
('Other noncurrent financial instruments', 0.1485148514851485)
('Deferred tax assets', 0.08888888888888889)
('Financial receivables from third parties and related companies', 0.08661417322834646)
('Financial receivables from affiliated companies', 0.08620689655172414)
Derivative financial assets

('Other financial assets', 0.5)
('Other noncur

In [235]:
for i, target in enumerate(targetDF):
    tarNgram = nGramCharTransformer(target)
    score = jaccardCoefficient(src1, tarNgram)
    score2= tverskyIndex(src1, tarNgram)
    print("="*50)
    print("{} \n{}\n{}\n{} -- {}".format(i, score, score2, sourceDf[0], target))

0 
0.04
0.07692307692307693
Current assets -- Trade payables to affiliated related companies
1 
0.021739130434782608
0.0425531914893617
Current assets -- Trade receivables from third parties
2 
0.08333333333333333
0.15384615384615385
Current assets -- Land and land rights
3 
0.2608695652173913
0.41379310344827586
Current assets -- Income tax assets
4 
0.058823529411764705
0.1111111111111111
Current assets -- Other short term deposits
5 
0.0
0.0
Current assets -- Other financial liabilities  employees
6 
0.0
0.0
Current assets -- VAT domestic
7 
0.11538461538461539
0.20689655172413793
Current assets -- Advanced payments
8 
0.021739130434782608
0.0425531914893617
Current assets -- Financial liabilities to shareholders
9 
0.0
0.0
Current assets -- Goodwill
10 
0.10416666666666667
0.18867924528301888
Current assets -- Restricted cash excluded from cash and cash equivalents
11 
0.125
0.2222222222222222
Current assets -- Prepayments to suppliers
12 
0.1111111111111111
0.2
Current assets -- R

## n-gram (word level) recommender (Jaccard)

## TF matching

## TF-IDF matching

## Levenshtein distance

## LSA factorization

## SVM

## word2vec